In [ ]:
import torch
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv
from huggingface_hub import login
import os
import pandas as pd

# Retrieval-Augmented Generation (RAG) Using Google Gemma-7b-it

In this notebook we will implement the following RAG architecture to create a customized LLM using text from my textbook **Handbook of Regression Modeling** and Google's Gemma-7b-it open source LLM:

![RAG Architecture](rag_architecture.jpg) 

We will be building two components, an information-retrieval (IR) component and a generation component:
* The IR component acts as a knowledge database of text files.  This database will be used to identify documents or text passages most relevant to the intent of the user query. Embeddings will be used to search this database and return the most relevant passages from the textbook.
* The generation component will feed the results of the IR component into the Gemma LLM as context in order to generate an attempt at a comprehensive natural language response to the user prompt.

## Preparing a dataset from my textbook to work with Gemma-7b-it LLM

My textbook is available open source, and its codebase is in an open Github repository.  We will connect to the repository and download the text of the 14 chapters and sections of the textbook, and create a Pandas dataframe with 14 rows and two columns, containing an ID-number for each chapter/section and the text of each chapter/section. 

In [ ]:
import requests

# chapters are Rmd files with the following names
chapter_list = [
    "01-intro",
    "02-basic_r",
    "03-primer_stats",
    "04-linear_regression",
    "05-binomial_logistic_regression",
    "06-multinomial_regression",
    "07-ordinal_regression",
    "08-hierarchical_data",
    "09-survival_analysis",
    "10-tidy_modeling",
    "11-power_tests",
    "12-further",
    "13-solutions",
    "14-bibliography"
]

# create a function to obtain the text of each chapter
def get_text(chapter: str) -> str:
    # URL on the Github where the rmd files are stored
    github_url = f"https://raw.githubusercontent.com/keithmcnulty/peopleanalytics-regression-book/master/r/{chapter}.Rmd"  
    
    result = requests.get(github_url)
    return result.text

# iterate over the chapter URLs and pull down the text content    
book_text = []
for chapter in chapter_list:
    chapter_text = get_text(chapter)
    book_text.append(chapter_text)

In [ ]:
# write to a dataframe
book_data = dict(chapter = list(range(14)), text = book_text)
book_data = pd.DataFrame.from_dict(book_data)

Most of these text documents are too long to fit into Gemma's context window, and so we will need to split them into smaller documents in a way that makes some sort of semantic sense.  

We will use a Langchain transformer to do semantic splitting, with a chunk size of 1000 and a chunk overlap of 150. 

In [ ]:
# semantically split chapters to a max length of 1000
loader = DataFrameLoader(book_data, page_content_column="text")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)

# examine a document to ensure it looks as we expect
docs[0]

## Generating embeddings and storing in a vector DB

![Embeddings](embeddings.png)

Since we will use embeddings for the IR component, we need to generate embeddings for our split dataset and then write those into a vector database to allow them to be searched.  We will use the *all-MiniLM-L6-v2 model* to generate the embeddings and the ChromaDB vector database to store them.  Vector Databases have limits to the number of documents that can be encoded in a single command, and so we will use a batch command just in case there are too many documents.

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.utils.batch_utils import create_batches
import uuid

In [ ]:
# set up the ChromaDB
CHROMA_DATA_PATH = "./chroma_data_regression_book/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "regression_book_docs"

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [ ]:
# enable the DB using Cosine Similarity as the distance metric
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func,
    metadata={"hnsw:space": "cosine"},
)

In [ ]:
# write text chunks to DB in batches 
batches = create_batches(
    api=client,
    ids=[f"{uuid.uuid4()}" for i in range(len(docs))], 
    documents=[doc.page_content for doc in docs], 
    metadatas=[{'source': './handbook_of_regression_modeling', 'row': k} for k in range(len(docs))]
)

for batch in batches:
    print(f"Adding batch of size {len(batch[0])}")
    collection.add(ids=batch[0],
                   documents=batch[3],
                   metadatas=batch[2])

Now our documents are persisting in our vector database, we can try running a query against them.

In [ ]:
results = collection.query(
    query_texts=["Which method would you recommend for ordered category outcomes?"],
    n_results=3, 
    include=['documents']
)

results

We can also see that our language model creates an embedding of 384 dimensions for each document.

## Pipelining the RAG using ChromaDB and Gemma-7b

We now have our vector DB in place so our IR layer is complete.  Now we will load the Gemma-7b LLM via Huggingface.  An access token is needed for this.  This model is large and will take some loading time.

In [ ]:
load_dotenv()
login(token=os.getenv("HF_TOKEN"))

In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS not available")

In [ ]:
# load model to Apple Silicon
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it", padding=True, truncation=True, max_length=512)

Now we have loaded the Gemma-7b-it model, we can set up an LLM pipeline.

We can run the model normally and ask a general question to the Gemma model:

In [ ]:
prompt = """
<start_of_turn>user
What food should I try in New Mexico?<end_of_turn>
<start_of_turn>model
"""

# embed the prompt
input_ids = tokenizer(prompt, return_tensors="pt")

# generate the answer
outputs = model.generate(**input_ids, max_new_tokens=512)

# decode the answer
tokenizer.decode(outputs[0], skip_special_tokens=True).split('model\n', 1)[1]

Finally, we define a function that executes our IR layer and our LLM summarization layer.  The function accepts a question and queries it against our ChromaDB, retrieving a defined number of documents based on the smallest distance from the query.  The results are joined together and sent to the LLM as context along with the original question, to generate a summarized result.

In [ ]:
def ask_question(question: str, model: AutoModelForCausalLM = model, tokenizer: AutoTokenizer = tokenizer, collection: str = COLLECTION_NAME, n_docs: int = 3) -> str:
    
    # Find close documents in chromadb
    collection = client.get_collection(collection)
    results = collection.query(
       query_texts=[question],
       n_results=n_docs
    )

    # Collect the results in a context
    context = "\n".join([r for r in results['documents'][0]])

    prompt = f"""
    <start_of_turn>user
    You are an expert on statistics and its applications to People Analytics.  
    Here is a question: {question}\n\n Answer it with reference to the following information and only using the following information: {context}.<end_of_turn>
    <start_of_turn>model
    """

    # Generate the answer using the LLM
    input_ids = tokenizer(prompt, return_tensors="pt")

    # Return the generated answer
    outputs = model.generate(**input_ids, max_new_tokens=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split('model\n', 1)[1]

## Testing our RAG agent

We test our agent by asking some questions where relevant information is known to exist in the IR layer.

In [ ]:
ask_question("What method would you recommend I use to model ordered category outcomes and why?")

In [ ]:
ask_question('What should I look out for when using Proportional Odds regression?')

In [ ]:
ask_question('What type of modeling is most likely to add value in People Analytics?')

In [ ]:
ask_question('Can you please explain what is meant by the term "inferential modeling"?')

In [ ]:
ask_question('Where did the term regression originate from?')

In [ ]:
ask_question('How do I get started using R for regression modeling?')

In [ ]:
ask_question('What factors determine statistical power?')

We also test to ensure that questions are only answered based on content in the book.

In [ ]:
ask_question('What is the standard model of Physics?')